###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2021 Semester 1

## Assignment 1: Pose classification with naive Bayes

###### Submission deadline: 7 pm, Tuesday 6 Apr 2021

**Student ID(s):**     `PLEASE ENTER YOUR ID(S) HERE`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing

def preprocess(train_path, test_path):
    headers = ['Pose', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10', 'y11']
    df_train = pd.read_csv(train_path, names=headers)
    df_test = pd.read_csv(test_path, names=headers)

    # replace 9999 with NaN for further processing
    df_train = df_train.replace([9999], np.nan)
    df_test = df_test.replace([9999], np.nan)

    '''Dealing with NaN values'''
    # For train data
    # filling missing values by median in each group (retain skewness)
    f = lambda x: x.median() if np.issubdtype(x.dtype, np.number) else x.mode().iloc[0]
    df_train = df_train.fillna(df_train.groupby('Pose').transform(f))

    # For test data
    # filling missing values by median of the whole dataset
    df_test = df_test.fillna(df_test.median())
    df_test = np.array(df_test)

    return df_train, df_test


train_path = 'data/train.csv'
test_path = 'data/test.csv'

df_train, df_test = preprocess(train_path, test_path)


In [4]:
# This function should calculate prior probabilities and likelihoods from the training data and usingthem to build a naive Bayes model

def train():
    # get all concepts(poses) as a list
    poses = list(df_train['Pose'].value_counts().index)

    # calculate statistics in training dataset
    pose_dict = dict()
    for pose in poses:
        pb_c = df_train.groupby('Pose').size()[pose] / len(df_train) # get prior probabilities
        pose_dict[pose] = np.array([pb_c, df_train[df_train['Pose'] == pose].describe().iloc[[1,2]].values], dtype=object) # get mean & stdv for all x and y (from 1 to 11)

    return pose_dict



pose_dict = train()

In [5]:
df_train[df_train['Pose'] == 'mountain'].describe().iloc[[1,2]]

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11
mean,-0.007176,-0.108334,-43.016699,-50.895127,43.169189,51.108017,-0.150617,-16.732309,-13.959189,16.623688,...,109.927956,64.608528,29.111031,64.743256,28.939961,9.954995,-72.358276,-159.434010,-72.369811,-159.565578
std,2.942956,2.257785,7.989678,18.969064,7.970678,19.072963,1.330744,3.742915,7.174475,3.709626,...,16.102933,15.385883,42.705678,15.185203,42.133322,16.103028,16.110207,17.872827,16.025912,17.942485


In [6]:
pose_dict['mountain']

array([0.214190093708166,
       array([[-7.17589375e-03, -1.08333981e-01, -4.30166994e+01,
        -5.08951269e+01,  4.31691888e+01,  5.11080169e+01,
        -1.50617486e-01, -1.67323087e+01, -1.39591893e+01,
         1.66236881e+01,  1.39685612e+01,  1.56441937e+02,
         1.09927956e+02,  6.46085281e+01,  2.91110312e+01,
         6.47432563e+01,  2.89399613e+01,  9.95499481e+00,
        -7.23582756e+01, -1.59434010e+02, -7.23698113e+01,
        -1.59565578e+02],
       [ 2.94295594e+00,  2.25778535e+00,  7.98967825e+00,
         1.89690637e+01,  7.97067830e+00,  1.90729632e+01,
         1.33074412e+00,  3.74291500e+00,  7.17447538e+00,
         3.70962646e+00,  6.98133847e+00,  1.95825884e+01,
         1.61029329e+01,  1.53858832e+01,  4.27056775e+01,
         1.51852034e+01,  4.21333225e+01,  1.61030279e+01,
         1.61102068e+01,  1.78728270e+01,  1.60259117e+01,
         1.79424848e+01]])], dtype=object)

In [7]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you can re-use the training data as a test set)

def calc_phi(x, mu, sigma):
    # return the likelihood of feature x_i in class c
    phi = 1 / (sigma * np.sqrt(2 * np.pi)) * np.e ** (- 1 / 2 * ((x - mu) / sigma) ** 2)

    # smoothing (but accuracy doesnt change tho)
    if phi == 0:
        phi = 0.000000001
    return phi

def predict():
    results = list()
    for instance in df_test:
        # prob_dict to store all the probabilities (log form) for comparison later
        prob_dict = dict()
        features = instance[1:] # remove concept
        for pose in pose_dict:
            prob_sum = 0
            pb_c = pose_dict[pose][0] # Prior probability of class c_j
            prob_sum += np.log(pb_c)
            for i in range(len(features)):
                x = features[i] # data x of feature x_i
                mu = pose_dict[pose][1][0][i] # mean of feature x_i
                sigma = pose_dict[pose][1][1][i] # stf of feature x_i
                pb_x_c = pb_c * calc_phi(x, mu, sigma) 
                prob_sum += np.log(pb_x_c)
            prob_dict[pose] = prob_sum
        predicted = max(prob_dict, key = prob_dict.get) # choose the highest value as predicted result
        actual = instance[0]
        results.append([predicted, actual])

    return results

results = predict()


In [8]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground truth labels

def evaluate():
    t = 0
    n = len(results)
    for predicted, actual in results:
        if predicted == actual:
            t += 1

    accuracy = t / n

    return accuracy

accuracy = evaluate()
print(accuracy)

0.5258620689655172


In [14]:
d = {key: {key: 0 for key in list(pose_dict.keys())} for key in list(pose_dict.keys())}
for predicted, actual in results:
    d[predicted][actual] += 1
d

{'mountain': {'mountain': 27,
  'downwarddog': 3,
  'bridge': 1,
  'childs': 1,
  'tree': 0,
  'trianglepose': 0,
  'plank': 0,
  'warrior2': 0,
  'warrior1': 0,
  'seatedforwardbend': 0},
 'downwarddog': {'mountain': 0,
  'downwarddog': 3,
  'bridge': 0,
  'childs': 3,
  'tree': 0,
  'trianglepose': 0,
  'plank': 3,
  'warrior2': 0,
  'warrior1': 0,
  'seatedforwardbend': 0},
 'bridge': {'mountain': 0,
  'downwarddog': 1,
  'bridge': 8,
  'childs': 0,
  'tree': 0,
  'trianglepose': 0,
  'plank': 2,
  'warrior2': 0,
  'warrior1': 0,
  'seatedforwardbend': 1},
 'childs': {'mountain': 0,
  'downwarddog': 0,
  'bridge': 0,
  'childs': 0,
  'tree': 0,
  'trianglepose': 0,
  'plank': 0,
  'warrior2': 0,
  'warrior1': 0,
  'seatedforwardbend': 0},
 'tree': {'mountain': 3,
  'downwarddog': 0,
  'bridge': 0,
  'childs': 0,
  'tree': 6,
  'trianglepose': 0,
  'plank': 0,
  'warrior2': 1,
  'warrior1': 0,
  'seatedforwardbend': 0},
 'trianglepose': {'mountain': 0,
  'downwarddog': 5,
  'bridge':

In [20]:
print(poses)

['mountain', 'downwarddog', 'bridge', 'childs', 'tree', 'trianglepose', 'plank', 'warrior1', 'warrior2', 'seatedforwardbend']


In [19]:
from sklearn.metrics import confusion_matrix

poses = list(pose_dict.keys())
p = list()
a = list()
for predicted, actual in results:
    p.append(predicted)
    a.append(actual)

confusion_matrix(a, p, labels=poses)


array([[27,  0,  0,  0,  3,  0,  0,  0,  0,  0],
       [ 3,  3,  1,  0,  0,  5,  6,  0,  0,  0],
       [ 1,  0,  8,  0,  0,  2,  3,  0,  0,  0],
       [ 1,  3,  0,  0,  0,  6,  3,  0,  0,  0],
       [ 0,  0,  0,  0,  6,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  4,  0,  0,  0,  0],
       [ 0,  3,  2,  0,  0,  3,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  4,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  7,  0],
       [ 0,  0,  1,  0,  0,  7,  0,  0,  0,  1]])

## Questions 


If you are in a group of 1, you will respond to **two** questions of your choosing.

If you are in a group of 2, you will respond to **four** questions of your choosing.

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer should be submitted separately as a PDF.

### Q1
Since this is a multiclass classification problem, there are multiple ways to compute precision, recall, and F-score for this classifier. Implement at least two of the methods from the "Model Evaluation" lecture and discuss any differences between them. (The implementation should be your own and should not just call a pre-existing function.)

In [ ]:
pose_dict_index = dict()
i= 0
pose_num = len(set(pose_dict))
# mark each pose as an integer index
for pose in set(pose_dict):
    pose_dict_index[pose]=i
    i+=1
pose_dict_index

# calculate confusion matrix
m_eval = np.zeros((pose_num,pose_num),dtype = int)
for predicted,actual in results:
    m_eval[pose_dict_index[predicted]][pose_dict_index[actual]]+=1

TP = np.zeros((1,pose_num))
FP = np.zeros((1,pose_num))
FN = np.zeros((1,pose_num))
precision_array = np.zeros((1,pose_num))
recall_array = np.zeros((1,pose_num))
for i in range(pose_num):
    tp = m_eval[i][i]                                               #tp value
    fp = sum(m_eval[i][j] for j in range(pose_num) if not j == i)   #fp value
    fn = sum(m_eval[j][i] for j in range(pose_num) if not j == i)   #fn value
    # Macro-averaging
    TP[0][i]=tp
    FP[0][i]=fp
    FN[0][i]=fn
    precision_array[0][i] = tp/(tp+fp) if not tp +fp ==0 else 0          
    recall_array[0][i] = tp/(tp+fn) if not tp +fn ==0 else 0

precision_macro = precision_array.mean()
recall_macro = recall_array.mean()

# Micro-averaging
precision_micro  = TP.sum()/(TP.sum()+FP.sum())
recall_micro = TP.sum()/(TP.sum()+FN.sum())
print("Precision with Macro-averaging: ",precision_macro.mean())
print("Recall with Macro-averaging: ",recall_macro.mean())
print("F-score with Macro-averaging:" , 2*precision_macro*recall_macro/(precision_macro+recall_macro))
print("Precision with Micro-averagin: ", precision_micro)
print("Recall with Micro-averagin: ", recall_micro)
print("F-score with Micro-averaging:" , 2*precision_micro*recall_micro/(precision_micro+recall_micro))

### Q2
The Gaussian naıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in this dataset? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the classifier’s predictions.

### Q3
Implement a kernel density estimate (KDE) naive Bayes classifier and compare its performance to the Gaussian naive Bayes classifier. Recall that KDE has kernel bandwidth as a free parameter -- you can choose an arbitrary value for this, but a value in the range 5-25 is recommended. Discuss any differences you observe between the Gaussian and KDE naive Bayes classifiers. (As with the Gaussian naive Bayes, this KDE naive Bayes implementation should be your own and should not just call a pre-existing function.)

In [108]:

# get all concepts(poses) as a list
poses = list(df_train['Pose'].value_counts().index)

# calculate statistics in training dataset
pb_c_dict = dict()
for pose in poses:
    pb_c_dict[pose] = df_train.groupby('Pose').size()[pose] / len(df_train)


pose_kde_dict = dict()

SIGMA = 5
# df_test as np array
results = list()
for instance in df_test:
        # prob_dict to store all the probabilities (log form) for comparison later
        prob_dict = dict()
        features = instance[1:] # remove concept
        for pose in poses:
            prob_sum = 0
            pb_c = pb_c_dict[pose] # prior probability of class c_j
            prob_sum += np.log(pb_c)

            for i in range(len(features)):
                xi_array = np.array(df_train[df_train['Pose'] == pose].drop(['Pose'], axis=1).iloc[:,i]) # get instance of test set
                x_test = features[i] # data x of feature x_i as x_test
                phi = 0
                for j in range(len(df_train)):
                    xi = xi_array[i]
                    phi += calc_phi(x_test, xi, SIGMA) 
                phi = phi / len(df_train)
                prob_sum += np.log(phi)

            prob_dict[pose] = prob_sum

        predicted = max(prob_dict, key = prob_dict.get) # choose the highest value as predicted result
        actual = instance[0]
        results.append([predicted, actual])

In [116]:
t = 0
n = len(results)
for predicted, actual in results:
    if predicted == actual:
        t += 1

accuracy = t / n
accuracy

0.3793103448275862

### Q4
Instead of using an arbitrary kernel bandwidth for the KDE naive Bayes classifier, use random hold-out or cross-validation to choose the kernel bandwidth. Discuss how this changes the model performance compared to using an arbitrary kernel bandwidth.

### Q5
Naive Bayes ignores missing values, but in pose recognition tasks the missing values can be informative. Missing values indicate that some part of the body was obscured and sometimes this is relevant to the pose (e.g., holding one hand behind the back). Are missing values useful for this task? Implement a method that incorporates information about missing values and demonstrate whether it changes the classification results.

### Q6
Engineer your own pose features from the provided keypoints. Instead of using the (x,y) positions of keypoints, you might consider the angles of the limbs or body, or the distances between pairs of keypoints. How does a naive Bayes classifier based on your engineered features compare to the classifier using (x,y) values? Please note that we are interested in explainable features for pose recognition, so simply putting the (x,y) values in a neural network or similar to get an arbitrary embedding will not receive full credit for this question. You should be able to explain the rationale behind your proposed features. Also, don't forget the conditional independence assumption of naive Bayes when proposing new features -- a large set of highly-correlated features may not work well.